<a href="https://colab.research.google.com/github/thanit456/Optuna_tutorial/blob/master/optuna_tf_eager_effiecientnetb4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna
!pip install efficientnet

     |████████████████████████████████| 204kB 8.4MB/s 
     |████████████████████████████████| 1.1MB 15.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 9.9MB/s 
     |████████████████████████████████| 81kB 10.3MB/s 
     |████████████████████████████████| 112kB 42.5MB/s 
     |████████████████████████████████| 122kB 31.7MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=418afd6f334b260a5fc25d4612b011583bf772f6ffa7ed0f110cd4f265592a1a
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-1.5.0-cp36-none-any.whl size=276145 sha256=00a0b913041aea330e625b101f0677a691fe8b428ceee37481ee8d5328a2a359
  Stored in directory: /root/.cache/pip/wheels/3

In [13]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import optuna

N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 1

In [14]:
def efficientnetb4_teacher(input_shape = (250, 100, 3), num_classes = 5):
    # 285, 120 2.44 | 414, 163, 2.65
    # import sys
    # sys.path.append('model/')
    import efficientnet.tfkeras as efn
    backbone = efn.EfficientNetB4(input_shape = input_shape, weights='imagenet', include_top = False)
    pooler = tf.keras.layers.GlobalAveragePooling2D()(backbone.output)
    out = tf.keras.layers.Dense(num_classes)(pooler)
    softmax = tf.keras.layers.Activation('softmax')(out)

    model = tf.keras.models.Model(inputs = backbone.input, outputs = softmax)
    return model

In [15]:
def create_efficientnet_model(trial):
  num_classes = 2

  aspect_ratio = trial.suggest_uniform("aspect_ratio", 2.0, 3.5)
  width = trial.suggest_int("width", 50, 200)
  height = int(aspect_ratio * width)
  model = efficientnetb4_teacher(input_shape=(height, width, 3), num_classes=num_classes)
  return model

In [36]:
def create_efficientnet_optimizer(trial):
  learning_rate_fn = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=trial.suggest_loguniform('initial_learning_rate', 1e-5, 1e-2),
    decay_steps=trial.suggest_int('decay_steps', 300, 10000),
    decay_rate=trial.suggest_uniform('decay_rate', 0.1, 0.9),
    staircase=True
  ) 
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)
  # optimizer = tf.keras.mixed_precision.experimental.LossScaleOptimizer(optimizer, loss_scale='dynamic')
  return optimizer

In [42]:
def loss_function(model, images, labels, mode='eval'):
  logits = model(images, training=(mode=='training'))
  loss_value = tf.reduce_mean(
      tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
  )
  return loss_value, logits

@tf.function
def train_step(model, optimizer, images, labels):
  with tf.GradientTape() as tape:
    loss_value, logits = loss_function(model, images, labels, mode='training')

  batch_loss = loss_value / images.shape[0]

  grads = tape.gradients(loss_value, model.variables)
  optimizer.apply_gradients(zip(grads, model.variabels))

@tf.function
def test_step(model, images, labels):
  loss_value, logits = loss_function(model, images, labels)
  # batch_loss = loss_value / images.shape[0]
  return logits 
  

In [43]:
def learn(model, optimizer, dataset, mode='eval'):
  accuracy = tf.metrics.CategoricalAccuracy()

  for batch, (images, labels) in enumerate(dataset):
    if mode == 'training':
      train_step(model, optimizer, images, labels)
    else:
      logits = test_step(model, images, labels)
      accuracy(tf.argmax(logits, axis=1, output_type=tf.int32), tf.cast(labels, tf.int32))
  if mode == 'eval':
    return accuracy


In [44]:
def get_mnist():
    (x_train, y_train), (x_valid, y_valid) = mnist.load_data()
    x_train = x_train.astype("float32") / 255
    x_valid = x_valid.astype("float32") / 255

    y_train = y_train.astype("int32")
    y_valid = y_valid.astype("int32")

    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.shuffle(60000).batch(BATCHSIZE).take(N_TRAIN_EXAMPLES)

    valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
    valid_ds = valid_ds.shuffle(10000).batch(BATCHSIZE).take(N_VALID_EXAMPLES)
    return train_ds, valid_ds

In [45]:
def objective(trial):
    # Get MNIST data.
    train_ds, valid_ds = get_mnist()

    # Build model and optimizer.
    model = create_efficientnet_model(trial)
    optimizer = create_efficientnet_optimizer(trial)

    # Training and validating cycle.
    with tf.device("/gpu:0"):
        for _ in range(EPOCHS):
            learn(model, optimizer, train_ds, "train")

        accuracy = learn(model, optimizer, valid_ds, "eval")

    # Return last validation accuracy.
    return accuracy.result()

In [46]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[W 2020-06-19 10:03:38,299] Setting status of trial#0 as TrialState.FAIL because of the following error: ValueError("in user code:\n\n    <ipython-input-37-f81b72c6744b>:20 test_step  *\n        loss_value, logits = loss_function(model, images, labels)\n    <ipython-input-42-f817bbc9a626>:2 loss_function  *\n        logits = model(images, training=(mode=='training'))\n    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:927 __call__  **\n        outputs = call_fn(cast_inputs, *args, **kwargs)\n    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/network.py:719 call\n        convert_kwargs_to_constants=base_layer_utils.call_context().saving)\n    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/network.py:888 _run_internal_graph\n        output_tensors = layer(computed_tensors, **kwargs)\n    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:886 __call__\n        self.name)\n  

ValueError: ignored